Notebook for the GoogleDecimeterChallenge https://www.kaggle.com/competitions/smartphone-decimeter-2023


Also try to run it on google collab, works great only RAM is limited:
https://colab.research.google.com/github/Torbynator/GoogleDecimeterChallenge/blob/main/main.ipynb#scrollTo=TOn-Can4C0YP

#Only for google collab
from google.colab import drive
drive.mount('/content/drive')
!pip install  kaggle
!mkdir -p ~/.kaggle
!cp /content/drive/MyDrive/kaggle.json ~/.kaggle/
!kaggle competitions download -c smartphone-decimeter-2023
!unzip /content/smartphone-decimeter-2023.zip

In [1]:
#reading data
import numpy as np 
import pandas as pd
from tqdm import tqdm, trange
import os

INPUT_PATH = 'sdc2023/'



In [2]:


test_input_data = []
test_gt_data = []

#iterate over all data files and store them in the respective arrays

#load test data
test_files = os.listdir(INPUT_PATH + "test")

for folder in test_files:
    smartphones = os.listdir(INPUT_PATH + "test/"+folder)
    for smartphone in smartphones:
        file =  "/device_gnss.csv"
        #store data in list while dropping first and 41st column (string data)
        test_input_data.append(pd.read_csv(INPUT_PATH + "test/" +folder+"/"+smartphone + file, usecols=[i for i in range(58) if i not in [0,40]], dtype=np.float32).to_numpy(dtype=np.float32).tolist())


In [4]:
#load train data
MAX_SAMPLES = 40
train_files = os.listdir(INPUT_PATH + "train")
sample_count=0

train_input_data = []
train_gt_data = []

for folder in train_files:
    smartphones = os.listdir(INPUT_PATH + "train/"+folder)
    for smartphone in smartphones:
        files = os.listdir(INPUT_PATH + "train/"+folder+"/"+smartphone)
        for file in files:
            if file.endswith(".csv"):
                if sample_count >= MAX_SAMPLES:
                    break   
                if "gnss" in file:
                    #store data in list while dropping first and 41st column (string data), as well as (porbably mostly) empty columns
                    #also all data points with the same timestep are seen as features of one timestep of a sample
                    train_input_data.append(pd.read_csv(INPUT_PATH + "train/" +folder+"/"+ smartphone+ "/" + file, usecols=[i for i in range(58) if i not in [0,2,3,4,22,23,24,26,28,19,30,31,32,33,34,35,40]], dtype=float).to_numpy(dtype=float).tolist())
                elif "ground_truth" in file:
                    sample_count +=1
                    #store data in list while dropping first and 2nd column (string data),(porbably mostly) empty columns
                    train_gt_data.append(pd.read_csv(INPUT_PATH + "train/"+folder+"/" + smartphone+ "/" + file,  usecols=[i for i in range(9) if i not in [0,1]], dtype=float).to_numpy(dtype=float).tolist())
                    print(f"read in {sample_count} samples")


read in 1 samples
read in 2 samples
read in 3 samples
read in 4 samples
read in 5 samples
read in 6 samples
read in 7 samples
read in 8 samples
read in 9 samples
read in 10 samples
read in 11 samples
read in 12 samples
read in 13 samples
read in 14 samples
read in 15 samples
read in 16 samples
read in 17 samples
read in 18 samples
read in 19 samples
read in 20 samples
read in 21 samples
read in 22 samples
read in 23 samples
read in 24 samples
read in 25 samples
read in 26 samples
read in 27 samples
read in 28 samples
read in 29 samples
read in 30 samples
read in 31 samples
read in 32 samples
read in 33 samples
read in 34 samples
read in 35 samples
read in 36 samples
read in 37 samples
read in 38 samples
read in 39 samples
read in 40 samples


In [5]:
from sklearn.preprocessing import MinMaxScaler

#normalize data
scaler = MinMaxScaler()
for sample in train_input_data:
    sample = scaler.fit_transform(sample)


c:\Users\Torbe\OneDrive\Documents\Uni\Machine Learning in Mechanics\GoogleDecimeterChallenge\.venv\Lib\site-packages\sklearn\utils\_array_api.py:701: RuntimeWarning: All-NaN slice encountered
  return xp.asarray(numpy.nanmin(X, axis=axis))
c:\Users\Torbe\OneDrive\Documents\Uni\Machine Learning in Mechanics\GoogleDecimeterChallenge\.venv\Lib\site-packages\sklearn\utils\_array_api.py:718: RuntimeWarning: All-NaN slice encountered
  return xp.asarray(numpy.nanmax(X, axis=axis))
c:\Users\Torbe\OneDrive\Documents\Uni\Machine Learning in Mechanics\GoogleDecimeterChallenge\.venv\Lib\site-packages\sklearn\utils\_array_api.py:701: RuntimeWarning: All-NaN slice encountered
  return xp.asarray(numpy.nanmin(X, axis=axis))
c:\Users\Torbe\OneDrive\Documents\Uni\Machine Learning in Mechanics\GoogleDecimeterChallenge\.venv\Lib\site-packages\sklearn\utils\_array_api.py:718: RuntimeWarning: All-NaN slice encountered
  return xp.asarray(numpy.nanmax(X, axis=axis))
c:\Users\Torbe\OneDrive\Documents\Uni\Ma

In [6]:
import tensorflow as tf

#move all features with the same timestamp to one line 

for sample in range(len(train_input_data)):
    matches = 0
    print(f"sample {sample}")
    for step in range(len(train_input_data[sample])):
        if step != 0:
            if train_input_data[sample][step-matches][0] == train_input_data[sample][step-1-matches][0]:
                train_input_data[sample][step-1-matches] = np.concatenate((train_input_data[sample][step-1-matches], train_input_data[sample][step-matches]))
                train_input_data[sample].pop(step-matches)
                matches +=1


#pad input data
max_timesteps = max(max([len(sample) for sample in train_input_data]), max([len(sample) for sample in train_gt_data]))
max_features = max(max([len(feature) for sample in train_input_data for feature in sample]),max([len(feature) for sample in train_gt_data for feature in sample]))

for sample in train_input_data:
    sample.extend([[-1]*max_features]*(max_timesteps-len(sample)))
train_input_data_padded = [tf.keras.preprocessing.sequence.pad_sequences(sample,value=-1, padding="post", dtype=np.int32, maxlen=max_features) for sample in train_input_data]
train_input_data_padded = np.array(train_input_data_padded)

#del train_input_data

#pad ground truth data
max_timesteps_gt = max_timesteps
max_features_gt = max([len(feature) for sample in train_gt_data for feature in sample])

for sample in train_gt_data:
    sample.extend([[-1]*max_features_gt]*(max_timesteps_gt-len(sample)))

#convert to right data format
train_gt_data_padded = [tf.keras.preprocessing.sequence.pad_sequences(sample, value=-1,padding="post", dtype=np.int64, maxlen=max_features_gt) for sample in train_gt_data]
train_gt_data_padded = np.array(train_gt_data_padded)
#del train_gt_data


sample 0
sample 1
sample 2


In [ ]:

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Masking
from tensorflow.keras.backend import clear_session

clear_session()

#create model
model = Sequential()
model.add(Masking(mask_value=-1, input_shape=(train_input_data_padded.shape[1], train_input_data_padded.shape[2])))
model.add(tf.keras.layers.Normalization(axis=(1,2)))
model.add(LSTM(60, return_sequences=True))
model.add(Dense(7))

model.compile(optimizer='adam', loss='mean_squared_error')


c:\Users\Torbe\OneDrive\Documents\Uni\Machine Learning in Mechanics\GoogleDecimeterChallenge\.venv\Lib\site-packages\keras\src\layers\core\masking.py:47: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


In [ ]:

print(train_input_data_padded.shape)
print(train_gt_data_padded.shape)

(2, 1303, 5166)
(2, 1303, 7)


In [ ]:

# train the model
model.fit(train_input_data_padded, train_gt_data_padded, epochs=10, batch_size=32, validation_split=0.2)


Epoch 1/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 5s 5s/step - loss: 362263321043798417473536.0000 - val_loss: 362263753389362645041152.0000
Epoch 2/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 1s/step - loss: 362263321043798417473536.0000 - val_loss: 362263753389362645041152.0000
Epoch 3/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 1s/step - loss: 362263321043798417473536.0000 - val_loss: 362263753389362645041152.0000
Epoch 4/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 1s/step - loss: 362263321043798417473536.0000 - val_loss: 362263753389362645041152.0000
Epoch 5/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 1s/step - loss: 362263321043798417473536.0000 - val_loss: 362263753389362645041152.0000
Epoch 6/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 1s/step - loss: 362263321043798417473536.0000 - val_loss: 362263753389362645041152.0000
Epoch 7/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 1s/step - loss: 362263321043798417473536.0000 - val_loss: 362263753389362645041152.0000
Epoch 8/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 1s/step - loss: 362263321043798417473536.0000 - val_loss: 36226375338936